# Preparation

Make sure you've followed all instructions listed in the `./README.md`. That should include:
1. Installing dependencies
2. Downloading train, dev and test data
3. Downloading the pre-trained model (Optionally you could train it yourself)

## Embeddings

In [1]:
import os
from scripts.fetch_embeddings import download_unzip

EMBEDDINGS_PATH = "./data/embeddings/"

GLOVE_URL = "http://www-nlp.stanford.edu/data/glove.840B.300d.zip"

if not os.path.exists(EMBEDDINGS_PATH):
    os.makedirs(EMBEDDINGS_PATH)

print(20*"=", "Fetching the word embeddings:", 20*"=")
download_unzip(GLOVE_URL, EMBEDDINGS_PATH)

==================== Fetching the word embeddings: ====================
* Found unzipped data in ./data/embeddings/, skipping download and unzip...


# Preprocessing

In [2]:
from scripts.preprocessing.preprocess import preprocess

In [3]:
DATA_PATH = "./data/dataset"
EMBEDDINGS_FILE_PATH = os.path.join(EMBEDDINGS_PATH, "glove.840B.300d.txt")
TARGET_PATH = "./data/preprocessed"

preprocess(
    DATA_PATH,
    EMBEDDINGS_FILE_PATH,
    TARGET_PATH,
    lowercase=False,
    ignore_punctuation=False,
    num_words=None,
    stopwords=[],
    labeldict= { "1": 1, "0": 0 },
    bos="_BOS_",
    eos="_EOS_",
)

====================  Preprocessing train set  ====================
	* Reading data...
	* Computing worddict and saving it...
	* Transforming words in premises and hypotheses to indices...
	* Saving result...
====================  Preprocessing dev set  ====================
	* Reading data...
	* Transforming words in premises and hypotheses to indices...
	* Saving result...
====================  Preprocessing test set  ====================
	* Reading data...
	* Transforming words in premises and hypotheses to indices...
	* Saving result...
====================  Preprocessing embeddings  ====================
	* Building embedding matrix and saving it...
Missed words:  27492


# Testing

In [4]:
from scripts.testing.test import main as test

In [5]:
PREPROCESSED_TEST_DATA_PATH = os.path.join(TARGET_PATH, "test_data.pkl")
CHECKPOINT_PATH = "./data/checkpoints/best.pth.tar"
test(PREPROCESSED_TEST_DATA_PATH, CHECKPOINT_PATH, "predictions.csv")

====================  Preparing for testing  ====================
	* Loading test data...
	* Building model...
====================  Testing ESIM model on device: cpu  ====================


Processing batches:   0%|          | 0/104 [00:00<?, ?it/s]/Users/kmilicic/Coding/University_of_Manchester/natural_language_understanding/ESIM/src/utils.py:44: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than tensor.new_tensor(sourceTensor).
  sequences_lengths.new_tensor(torch.arange(0, len(sequences_lengths))).to(device)
Processing batches: 100%|██████████| 104/104 [00:06<00:00, 16.93it/s]

-> Average batch processing time: 0.0584s, total test time: 6.1701s


In [6]:
PREPROCESSED_DEV_DATA_PATH = os.path.join(TARGET_PATH, "dev_data.pkl")
test(PREPROCESSED_DEV_DATA_PATH, CHECKPOINT_PATH, "dev_predictions.csv")

====================  Preparing for testing  ====================
	* Loading test data...
	* Building model...
====================  Testing ESIM model on device: cpu  ====================


Processing batches: 100%|██████████| 211/211 [00:19<00:00, 10.87it/s]

-> Average batch processing time: 0.0914s, total test time: 19.4268s


# Evaluation

Performing evaluation only od dev set as we don't have access to the ground truth labels for the test set.

In [7]:
from scripts.evaluation.evaluate import main as evaluate

TEST_DATA_PATH = "./data/dataset/dev.csv"
PREDICTIONS_PATH = "./data/results/dev_predictions.csv"
evaluate(TEST_DATA_PATH, PREDICTIONS_PATH)

Accuracy: 70.11%
Macro Precision: 70.42%
Macro Recall: 69.85%
Macro F1 Score: 69.79%
Weighted Precision: 70.36%
Weighted Recall: 70.11%
Weighted F1 Score: 69.89%
Matthews Correlation Coefficient (MCC): 40.27%
